RetinaNet的介绍与升级
===

# 1.RetinaNet的介绍
- RetinaNet是一种介于一阶段与二阶段检测器，它综合了两个方法的优点
- 无论是一阶段还是二阶段检测算法, 本质上都会具有两个分支, 一个用来回归, 一个用来分类。唯一的区别只在于, 二阶段算法具备一个RPN的模块, 也就是我们常说的, Region Proposal. 在这里进行二分类, 把大部分背景干掉. 

RetinaNet其实跟SSD的结构几乎一模一样, 只不过它加了FPN, 我们先来看看SSD结构是啥样的:
![Images](images/04_01_001.jpg)

总结这个one stage核心架构如下:
- 它从骨干网络中, 若干个尺度的featuremaps提取特征;
- 这些特征将分别进入到loc model和cls model回归坐标和类别;
- 训练时通过预先设定的Anchor或者说是PriorBox对GT进行编码, 对网络输出进行回归;
- 预测时, 对网络的输出进行解码, 得到真实尺寸, 通过nms之后得到最终检测结果.

整个过程非常流畅，比FasterRCNN简单很多. 我们需要做的就是加入FPN。SSD使用的是融合特征的思想去做检测优化，而FPN似乎在这个基础上更进一步, 它不仅仅做特征融合, 还引入了一个编码和解码的思想

# 2.FPN的实现

![Images](images/04_01_002.jpg)

FPN 也就是特征金字塔, 总结起来可以如下:
- 这个玩意儿不是单纯的尺度变小然后挨个分支出来(SSD的办法), 而是先压缩, 再增大, 然后同层级的进行融合;
- 整体结果相对来说比较简单.

![Images](images/04_01_003.jpg)

从这张图我们可以看出网络的脉络和数据流:
- 首先我们从backbone的最后三层拿到featuremaps, 这三层如何选择呢, 通常情况下我们采用尺度诸如128,256,512大小的featuremap, 足够大才能保留更多的分辨率;
- 然后我们经过一个从地向上, 然后从上到下的结果, 用专业术语来讲就是bottom-up and top-down的结构. 经过一些复杂的操作, 我们最终可以拿到我们需要的柔和之后的featuremaps;
- 事实正面, 这种融合之后的特征可以大大的增强我们的检测器精度.

# 3.Regression Part

可以跟SSD保持一致，只要保持输出$numpriors \times 4 \times w \times h$就可以了

# 4.Focus Loss

- 防止detector被训废掉. 这在一阶段检测网络里面很普遍, 网络跑着跑着就不收敛了, 最后效果不好. 这是因为对背景过拟合了. focal loss可以让背景这样的好预测的样本对loss的贡献变小, 从而让网络花更多精力去学习难以预测的正常目标;
- 自动化的解决一阶段检测器正负样本不均衡的问题.